# Infos sur la photométrie 

- auteur : Sylvie Dagoret-Campagne
- date de creation : 3 février 2021
- affiliation : IJCLab/IN2P3/CNRS

- https://docs.astropy.org/en/stable/units/

- https://docs.astropy.org/en/stable/constants/

- https://docs.astropy.org/en/stable/units/equivalencies.html

- https://learn.astropy.org/tutorials.html

- https://learn.astropy.org/rst-tutorials/UVES.html?highlight=filtertutorials

- https://docs.astropy.org/en/stable/cosmology/

## Import


In [48]:
from astropy import constants as const

In [49]:
#from astropy import physical_constants, astronomical_constants

In [50]:
from astropy import units as u

In [51]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Flux


Par définition le flux de réference donnant une magnitude AB nulle est
$$
F_{ref} = 3631.1 Jy 
$$

avec
$$
1 Jy = 10^{-26} W.Hz^{-1}.m^{-2} = 10^{-23}.erg.cm^{-2} s^{-1} . Hz^{-1}
$$

## Flux de référence

In [52]:
Fref=3631.1 * u.Jy

In [53]:
target_flux = Fref

In [54]:
zero_point_star_equiv = u.zero_point_flux(3631.1 * u.Jy)

In [55]:
zero_point_star_equiv

[(Unit("mgy"), Unit("3631.1 Jy"))]

https://docs.astropy.org/en/stable/units/equivalencies.html

In [56]:
Fref.to(u.W / (u.m)**2 / u.Hz,equivalencies=u.spectral_density(550 * u.nm))  

<Quantity 3.6311e-23 W / (Hz m2)>

In [57]:
Fref.to(u.W / (u.m)**2 / u.Hz,equivalencies=u.spectral_density(800 * u.nm))  

<Quantity 3.6311e-23 W / (Hz m2)>

In [58]:
# so zero magnitude by definition
u.Magnitude(target_flux.to(u.AB, zero_point_star_equiv))  

<Magnitude -9.55237213e-05 mag(AB)>

In [59]:
u.Magnitude(target_flux.to(u.AB, Fref))  

<Magnitude -9.55237213e-05 mag(AB)>

# Flux

$$
C_b = \frac{S_{coll}.\Delta T}{g_{el} h} \int_0^\infty F_\nu(\lambda).S^{atm}(\lambda).S^{syst}_b(\lambda) \frac{d\lambda}{\lambda}
$$

- $C_b$ : taux ADU par seconde
- $S_{coll}$ : surface collectrice du telescope
- $\Delta T$ : temps d'exposition de l'observation
- $F_\nu(\lambda)$ : flux spécifique en $W/Hz/m^2$ de la source au dessus de l'atmosphère
- $S^{atm}(\lambda)$ : transmission de l'atmosphère, variable dans le temps et selon l'inclaison de l'observation
- $S^{syst}_b(\lambda)$ : transmission du télescope (transmission des mirroirs, transmission des lentilles correctrices, efficacité quantique du CCD (du photon au photoelectron) dans une bande passante $b$ à support limité en fréquence


Notons que le flux est souvent exprimé en Jansky, une unité non SI:

$
1 Jy = 10^{-26} W/Hz/m^2 = 10^{-23} erg/cm^2/s/Hz
$

## Constantes

En unités ADU/s

$$
C = \frac{\pi D^2 \Delta t}{4 g_{el} h}
$$

In [60]:
D=6.5*u.m # diametre du telescope
Dt=30*u.s # durée de l'exposition par image
gel=1.    # nombre de photoelectron par ADU

In [61]:
C = np.pi*D**2*Dt/4/const.h/gel

In [62]:
C

<Quantity 1.50238701e+36 m2 / J>

In [63]:
C.decompose()

<Quantity 1.50238701e+36 s2 / kg>

## Définition  du flux de référence pour la magnitude AB
$$
F_\nu(\lambda) = F_0 . f_\nu(\lambda)
$$

avec

- $F_0$ : en W/Hz/$m^2$,

- $f_\nu(\lambda)$ : sans dimension



$$
m_{0}^{AB} = -2.5 \log_{10} \left( \frac{F_0}{3631 Jy}\right)
$$

$$
F_\nu(\lambda) = 3631 Jy \cdot 10^{-0.4 m_0^{AB}} \cdot f_\nu(\lambda)
$$

Pour une source de magnitude caractéristique de 25 mag:

$$
F_\nu(\lambda) =  3.63 10^{-13}{\rm W/Hz/m^2}\cdot 10^{0.4 (25-m_0^{AB})} \cdot f_\nu(\lambda)
$$


### Vérification dans s'embarrasser avec les unités:

- on se place dans le SI

In [64]:
# Conversion de 1Jy en W/Hz/m2 et on divise par 1O^0.4*25
# Calcul dans le SI
K0=3631.1*1e-26*np.power(10,-10.)
K0

3.6311e-33

In [65]:
# Pour une surface collectrice correspondant à D=6.5 m et Delta t = 30 sec
# K1 est le terme de photoelectrons produits par le CCD pendant Delta T= 30 secondes
# Calcul dans le SI
K1=K0*np.pi*6.5**2/4*30./6.626e-34
K1

5455.375228093965

### Vérification avec le SI

In [66]:
Cb0=C*Fref
Cb0

<Quantity 5.45531747e+39 Jy m2 / J>

In [67]:
Cb0.decompose()

<Quantity 5.45531747e+13>

In [68]:
C0=np.power(10,0.4*25)

In [69]:
Cb0.decompose()/C0

<Quantity 5455.3174722>

## Comptage dans l'instrument

- $C_b$ : nombre d'ADU (unités d'ADC)
- $N_{pe} = C_b . g_{el}$ : nombre de photoelectrons produits dans le CCD 

$$
C_b = \frac{5455}{g_{el}}. 10^{0.4(25-m_{AB})}. \left( \frac{D}{6.5m} \right)^2 \left( \frac{\Delta t}{30 sec}\right) \int_0^\infty f_\nu(\lambda).S^{atm}(\lambda).S^{syst}_b(\lambda)\frac{d\lambda}{\lambda}
$$